In [4]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets
from roboflow import Roboflow
rf = Roboflow(api_key=" YOUR_KEY")
project = rf.workspace("wilgvalz").project("game_unity_detect")
version = project.version(1)
dataset = version.download("yolov7")

mkdir: cannot create directory ‘{HOME}/datasets’: No such file or directory
[Errno 2] No such file or directory: '{HOME}/datasets'
/content
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Game_unity_Detect-1 in yolov7pytorch:: 100%|██████████| 294/294 [00:00<00:00, 4328.26it/s]


In [ ]:
#Обучение SDD модели на датасете для Object Detection. Датасет собран вручную.
#Содержит 5 классов (мною были выбраны 5 жестов рук), разметка
#bounding boxes проведена при помощи roboflow
#цель - демонстрационная, а не высокая точность

In [13]:
import os
os.listdir('/content/Game_unity_Detect-1/train/images')[0]

'no-98d5ec00-459b-11ef-8594-9801a7964a15_jpg.rf.ef486ff24c22578a73add9eedc04bbb8.jpg'

In [11]:
imgs = os.listdir('/content/Game_unity_Detect-1/train/images')

In [54]:
import re
idx = []
for i in range(len(imgs)):
  idx.append(re.match('.*?(?=\.jpg)', imgs[i]).group(0))

In [55]:
idx

['no-98d5ec00-459b-11ef-8594-9801a7964a15_jpg.rf.ef486ff24c22578a73add9eedc04bbb8',
 'no-9c976684-459b-11ef-8594-9801a7964a15_jpg.rf.ad4ca59f59963f81149a3bde2ae7ad68',
 'iloveyou-adefd920-459b-11ef-8594-9801a7964a15_jpg.rf.3bcc96a9c1bf9e7a693c1f3cf947fb9c',
 'hello-5721e32c-459b-11ef-8594-9801a7964a15_jpg.rf.111ac037aacccfc65fe0003bf2e1b53f',
 'yes-83c24ff2-459b-11ef-8594-9801a7964a15_jpg.rf.a8eaf93d71a14f5ee8d8c8bfc27cecd9',
 'yes-88c1b79a-459b-11ef-8594-9801a7964a15_jpg.rf.29f1f0b810a3a41de9425ca05a12913a',
 'yes-83c24ff2-459b-11ef-8594-9801a7964a15_jpg.rf.9fd4c2a21b9187f08ee20d5373774fcf',
 'thanks-6d786808-459b-11ef-8594-9801a7964a15_jpg.rf.e97dd67bf737754814e1052f82ca0789',
 'iloveyou-a7a85efc-459b-11ef-8594-9801a7964a15_jpg.rf.f01e9a5c48964f25ae9e4875c994219d',
 'iloveyou-a8e838aa-459b-11ef-8594-9801a7964a15_jpg.rf.fe229b589b299a71334135018238b50b',
 'thanks-6d786808-459b-11ef-8594-9801a7964a15_jpg.rf.9ef9c09f2b9a8427946869fb33aa9f98',
 'thanks-74fa00aa-459b-11ef-8594-9801a7964a1

In [60]:
data = []
iss = os.listdir('/content/Game_unity_Detect-1/train/labels')
for i in range(len(iss)):
  with open("/content/Game_unity_Detect-1/train/labels/"+ iss[i]) as f:

    for line in f:
        data.append([float(x) for x in line.split() if x != 1 and x!=0 and x!=2 and x!=3 and x!= 4])

In [66]:
data[0]

[2.0, 0.7234375, 0.49140625, 0.290625, 0.29375]

In [71]:
XMin = []
XMax = []
YMin = []
YMax = []
for i in range(len(data)):
  XMin.append(data[i][1])
  XMax.append(data[i][2])
  YMin.append(data[i][3])
  YMax.append(data[i][4])

In [68]:
len(XMax)

123

In [48]:
import re
res = []
for i in range(len(imgs)):
  res.append(re.match('.*?(?=\-)', imgs[i]).group(0))

In [73]:
df_cols

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,no-98d5ec00-459b-11ef-8594-9801a7964a15_jpg.rf...,no,0.723437,0.491406,0.290625,0.293750
1,no-9c976684-459b-11ef-8594-9801a7964a15_jpg.rf...,no,0.783594,0.242188,0.185938,0.289062
2,iloveyou-adefd920-459b-11ef-8594-9801a7964a15_...,iloveyou,0.503125,0.208594,0.181250,0.311719
3,hello-5721e32c-459b-11ef-8594-9801a7964a15_jpg...,hello,0.351562,0.545312,0.318750,0.535937
4,yes-83c24ff2-459b-11ef-8594-9801a7964a15_jpg.r...,yes,0.491406,0.763281,0.234375,0.340625
...,...,...,...,...,...,...
118,no-9515fc72-459b-11ef-8594-9801a7964a15_jpg.rf...,no,0.649219,0.897656,0.201563,0.198437
119,thanks-64b3df36-459b-11ef-8594-9801a7964a15_jp...,thanks,0.650000,0.537500,0.292187,0.523438
120,yes-864121ae-459b-11ef-8594-9801a7964a15_jpg.r...,yes,0.531250,0.671094,0.345313,0.614062
121,no-9294a66a-459b-11ef-8594-9801a7964a15_jpg.rf...,no,0.225781,0.703125,0.209375,0.314063


In [72]:
import pandas as pd
df_cols = pd.DataFrame({'ImageID': idx, 'LabelName': res, 'XMin': XMin, 'XMax': XMax, 'YMin': YMin, 'YMax': YMax})

In [99]:
from torch_snippets import *


label2target = {l:t+1 for t,l in enumerate(df_cols['LabelName'].unique())}
label2target['background'] = 0
target2label = {t:l for l,t in label2target.items()}
background_class = label2target['background']
num_classes = len(label2target)

device = 'cuda' if torch.cuda.is_available() else 'cpu'


import collections, os, torch
from PIL import Image
from torchvision import transforms
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)
denormalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.255],
    std=[1/0.229, 1/0.224, 1/0.255])

def preprocess_image(img):
    img = torch.tensor(img).permute(2,0,1)
    img = normalize(img)
    return img.to(device).float()

class OpenDataset(torch.utils.data.Dataset):
    w, h = 300, 300
    def __init__(self, df, image_dir='/content/Game_unity_Detect-1/train/images'):
        self.image_dir = image_dir
        self.files = glob.glob(self.image_dir+'/*')
        self.df = df
        self.image_infos = df.ImageID.unique()
        logger.info(f'{len(self)} items loaded')

    def __getitem__(self, ix):
        # load images and masks
        image_id = self.image_infos[ix]
        img_path = find(image_id, self.files)
        img = Image.open(img_path).convert("RGB")
        img = np.array(img.resize((self.w, self.h), resample=Image.BILINEAR))/255.
        data = df_cols[df_cols['ImageID'] == image_id]
        labels = data['LabelName'].values.tolist()
        data = data[['XMin','YMin','XMax','YMax']].values
        data[:,[0,2]] *= self.w
        data[:,[1,3]] *= self.h
        boxes = data.astype(np.uint32).tolist() # convert to absolute coordinates
        return img, boxes, labels
    def collate_fn(self, batch):
        images, boxes, labels = [], [], []
        for item in batch:
            img, image_boxes, image_labels = item
            img = preprocess_image(img)[None]
            images.append(img)
            boxes.append(torch.tensor(image_boxes).float().to(device)/300.)
            labels.append(torch.tensor([label2target[c] for c in image_labels]).long().to(device))
        images = torch.cat(images).to(device)
        return images, boxes, labels
    def __len__(self):
        return len(self.image_infos)


In [100]:
from sklearn.model_selection import train_test_split
trn_ids, val_ids = train_test_split(df_cols.ImageID.unique(), test_size=0.1, random_state=99)
trn_df, val_df = df_cols[df_cols['ImageID'].isin(trn_ids)], df_cols[df_cols['ImageID'].isin(val_ids)]
len(trn_df), len(val_df)

train_ds = OpenDataset(trn_df)
test_ds = OpenDataset(val_df)

train_loader = DataLoader(train_ds, batch_size=2, collate_fn=train_ds.collate_fn, drop_last=True)
test_loader = DataLoader(test_ds, batch_size=2, collate_fn=test_ds.collate_fn, drop_last=True)


[08/01/24 18:23:00] INFO     110 items loaded                                                                                    ]8;id=785477;file://<ipython-input-99-ce8baa879414>:40\<ipython-input-99-ce8baa879414>]8;;\:]8;id=874056;file://<ipython-input-99-ce8baa879414>:40#__init__:40\__init__:40]8;;\

                    INFO     13 items loaded                                                                                     ]8;id=30677;file://<ipython-input-99-ce8baa879414>:40\<ipython-input-99-ce8baa879414>]8;;\:]8;id=953818;file://<ipython-input-99-ce8baa879414>:40#__init__:40\__init__:40]8;;\

In [101]:
import glob
from torch_snippets import *

In [102]:
def train_batch(inputs, model, criterion, optimizer):
    model.train()
    N = len(train_loader)
    images, boxes, labels = inputs
    _regr, _clss = model(images)
    loss = criterion(_regr, _clss, boxes, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss

@torch.no_grad()
def validate_batch(inputs, model, criterion):
    model.eval()
    images, boxes, labels = inputs
    _regr, _clss = model(images)
    loss = criterion(_regr, _clss, boxes, labels)
    return loss


In [103]:
from model import SSD300, MultiBoxLoss
#from torch import SSD300, MultiBoxLoss
from detect import *
import torch
#model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd').cuda()

n_epochs = 3

model = SSD300(num_classes, device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = MultiBoxLoss(priors_cxcy=model.priors_cxcy, device=device)

log = Report(n_epochs=n_epochs)
logs_to_print = 5


The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.



Loaded base model.



size_average and reduce args will be deprecated, please use reduction='none' instead.


In [93]:
import os

!pip install -q torch_snippets
!git clone https://github.com/sizhky/ssd-utils/
%cd ssd-utils

Cloning into 'ssd-utils'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (9/9), 13.65 KiB | 2.28 MiB/s, done.
/content/ssd-utils


In [104]:
for epoch in range(n_epochs):
    _n = len(train_loader)
    for ix, inputs in enumerate(train_loader):
        loss = train_batch(inputs, model, criterion, optimizer)
        pos = (epoch + (ix+1)/_n)
        log.record(pos, trn_loss=loss.item(), end='\r')

    _n = len(test_loader)
    for ix,inputs in enumerate(test_loader):
        loss = validate_batch(inputs, model, criterion)
        pos = (epoch + (ix+1)/_n)
        log.record(pos, val_loss=loss.item(), end='\r')


image_paths = Glob(f'/content/Game_unity_Detect-1/train/images/*')
image_id = choose(test_ds.image_infos)
img_path = find(image_id, test_ds.files)
original_image = Image.open(img_path, mode='r')
original_image = original_image.convert('RGB')

EPOCH: 3.000  val_loss: 7.046  (1464.72s - 0.00s remaining)